In [8]:
import pandas as pd
import numpy as np
import data_cleaning
import chicago_api as chi

In [3]:
## Testing restaurant df filtering
chunk_size = 100000
restaurants_df = data_cleaning.filter_business_data("data/yelp_academic_dataset_business.json", chunk_size)

## Testing save and load
data_cleaning.save(restaurants_df, "data/filtered_restaurants.json")
reloaded_restaurants_df = data_cleaning.load("data/filtered_restaurants.json", chunk_size)
reloaded_restaurants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27894 entries, 0 to 27893
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   27894 non-null  object 
 1   name          27894 non-null  object 
 2   address       27894 non-null  object 
 3   city          27894 non-null  object 
 4   state         27894 non-null  object 
 5   postal_code   27894 non-null  object 
 6   latitude      27894 non-null  float64
 7   longitude     27894 non-null  float64
 8   stars         27894 non-null  float64
 9   review_count  27894 non-null  int64  
 10  is_open       27894 non-null  int64  
 11  attributes    27888 non-null  object 
 12  categories    27894 non-null  object 
 13  hours         26538 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 3.0+ MB


In [4]:
## Testing review df filtering
chunk_size = 100000
reviews_df = data_cleaning.filter_review_data("data/yelp_academic_dataset_review.json", restaurants_df, chunk_size)
reviews_df.info()

## Testing save and load
data_cleaning.save(reviews_df, "data/filtered_reviews.json")
reloaded_reviews_df = data_cleaning.load("data/filtered_reviews.json", chunk_size)
reloaded_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4371248 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 333.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4371248 entries, 0 to 4371247
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date        

In [18]:
## Testing users df filtering
chunk_size = 100000
users_df = filter_user_data("data/yelp_academic_dataset_user.json", reviews_df, chunk_size)
users_df.info()

## Testing save and load
save(users_df, "data/filtered_users.json")
reloaded_users_df = load("data/filtered_users.json", chunk_size)
reloaded_users_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1379968 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   user_id             1379968 non-null  object 
 1   name                1379968 non-null  object 
 2   review_count        1379968 non-null  int64  
 3   yelping_since       1379968 non-null  object 
 4   useful              1379968 non-null  int64  
 5   funny               1379968 non-null  int64  
 6   cool                1379968 non-null  int64  
 7   elite               1379968 non-null  object 
 8   friends             1379968 non-null  object 
 9   fans                1379968 non-null  int64  
 10  average_stars       1379968 non-null  float64
 11  compliment_hot      1379968 non-null  int64  
 12  compliment_more     1379968 non-null  int64  
 13  compliment_profile  1379968 non-null  int64  
 14  compliment_cute     1379968 non-null  int64  
 15  compliment_list     

In [9]:
## Pull Chicago API data
#chi_restaurants_df = chi.get_chicago_restaurants_df(0, 10)
#chi_restaurants_df = chi.get_chicago_restaurants_df(10, 60)
#chi_restaurants_df = chi.get_chicago_restaurants_df(60, 110)
#chi_restaurants_df = chi.get_chicago_restaurants_df(110, 160)
chi_restaurants_df = chi.get_chicago_restaurants_df(160, 210)

Near North Side has 5500 hits


/home/chasty2/Documents/group-project-better-late-than-never/chicago_api.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_restaurants = pd.concat([all_restaurants, restaurants_df], ignore_index=True)


Lincoln Park has 6000 hits
Norwood Park has 2500 hits
Logan Square has 1700 hits
Greater Grand Crossing has 634 hits
Lincoln Park has 6000 hits
Ashburn has 1900 hits
West Ridge has 1100 hits
Calumet Heights has 366 hits
Lower West Side has 5000 hits
Logan Square has 1700 hits
West Town has 6300 hits
Avondale has 6500 hits
Irving Park has 6200 hits
Portage Park has 891 hits
Near South Side has 2300 hits
Douglas has 929 hits
Roseland has 1300 hits
The Loop has 2800 hits
West Town has 6300 hits
Pullman has 260 hits
Lincoln Park has 6000 hits
Lincoln Square has 1800 hits
Lincoln Square has 1800 hits
Albany Park has 4900 hits
Near North Side has 5500 hits
West Town has 6300 hits
North Park has 1100 hits
Riverdale has 972 hits
Rogers Park has 814 hits
North Center has 631 hits
West Ridge has 1100 hits
Roseland has 1300 hits
Roseland has 1300 hits
North Center has 631 hits
Forest Glen has 523 hits
O'Hare has 1900 hits
Dunning has 2700 hits
Ashburn has 1900 hits
Lincoln Park has 6000 hits
Upto

In [10]:
all_chi_restaurants = data_cleaning.load("data/chicago_restaurants_incomplete", 100_000)

In [ ]:
all_chi_restaurants.info()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,business_hours,attributes,neighborhood
0,vhR0uUVBmAO2f7VfrxMUog,tepalcates-chicago,Tepalcates,https://s3-media2.fl.yelpcdn.com/bphoto/bES1Oc...,False,https://www.yelp.com/biz/tepalcates-chicago?ad...,8,"[{'alias': 'newmexican', 'title': 'New Mexican...",5.0,"{'latitude': 41.923839, 'longitude': -87.754963}",[],None,"{'address1': '5131 W Fullerton Ave', 'address2...",+17732376638,(773) 237-6638,5681.528850,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",Albany Park
1,EZL3IDddaN0RzxJxr6k6-Q,pizzeria-serio-event-space-chicago,Pizzeria Serio Event Space,https://s3-media3.fl.yelpcdn.com/bphoto/5KrSSG...,False,https://www.yelp.com/biz/pizzeria-serio-event-...,10,"[{'alias': 'venues', 'title': 'Venues & Event ...",5.0,"{'latitude': 41.939848, 'longitude': -87.671525}",[],None,"{'address1': '1708 W Belmont Ave', 'address2':...",+17735250600,(773) 525-0600,5203.186755,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht...",Albany Park
2,bPw46bO4gic_U1HOnlY34Q,la-flor-michoacana-3-chicago,la flor michoacana 3,https://s3-media3.fl.yelpcdn.com/bphoto/6N3c5w...,False,https://www.yelp.com/biz/la-flor-michoacana-3-...,7,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",5.0,"{'latitude': 41.9010187556, 'longitude': -87.6...",[],None,"{'address1': '1045 N Ashland Ave', 'address2':...",+17737998140,(773) 799-8140,8736.497724,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': Non...",Albany Park
3,o7axg6eu7wE_NJSfiaGqAw,sushi-by-bou-river-north-chicago-2,Sushi By Bou - River North Chicago,https://s3-media3.fl.yelpcdn.com/bphoto/SWyzjD...,False,https://www.yelp.com/biz/sushi-by-bou-river-no...,6,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",5.0,"{'latitude': 41.895179, 'longitude': -87.6343}",[],None,"{'address1': '712 N Wells St', 'address2': '',...",+18886705996,(888) 670-5996,10865.363993,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht...",Albany Park
4,k_r53_vc4sVwPYA38h6c_A,la-cocina-de-don-ramon-chicago,La Cocina de Don Ramon,https://s3-media2.fl.yelpcdn.com/bphoto/vhYeG2...,False,https://www.yelp.com/biz/la-cocina-de-don-ramo...,15,"[{'alias': 'breakfast_brunch', 'title': 'Break...",5.0,"{'latitude': 41.957255278, 'longitude': -87.75...","[delivery, pickup]",None,"{'address1': '4208 N Milwaukee Ave', 'address2...",+17086670881,(708) 667-0881,2807.510825,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",Albany Park


In [12]:
all_chi_restaurants = pd.concat([all_chi_restaurants, chi_restaurants_df], ignore_index=True)
data_cleaning.save(all_chi_restaurants, "data/chicago_restaurants_incomplete")

In [13]:
all_chi_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50148 entries, 0 to 50147
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              50148 non-null  object 
 1   alias           50148 non-null  object 
 2   name            50148 non-null  object 
 3   image_url       50148 non-null  object 
 4   is_closed       50148 non-null  object 
 5   url             50148 non-null  object 
 6   review_count    50148 non-null  object 
 7   categories      50148 non-null  object 
 8   rating          50148 non-null  float64
 9   coordinates     50148 non-null  object 
 10  transactions    50148 non-null  object 
 11  price           21745 non-null  object 
 12  location        50148 non-null  object 
 13  phone           50148 non-null  object 
 14  display_phone   50148 non-null  object 
 15  distance        50148 non-null  float64
 16  business_hours  50148 non-null  object 
 17  attributes      50148 non-null 